# Задание 1

### a)

$f(x) = c^Tx$

$h(x) = Ax - b$

$g(x) = -x$

$L(x, \lambda, \mu) = f(x) + \lambda^T g(x) + \mu^T h(x) = c^Tx - \lambda^Tx + \mu^T(Ax - b) = (c^T - \lambda^T + \mu^T A) x - \mu^Tb$

$q(\lambda, \mu) = \inf\limits_{x \in \mathbb{R}^n} L(x, \lambda, \mu) = \inf\limits_{x \in \mathbb{R}^n} ((c^T - \lambda^T + \mu^T A) x - \mu^Tb) = \begin{cases}
-\mu^Tb & c^T - \lambda^T + \mu^TA = 0\\
-\infty& c^T - \lambda^T + \mu^TA \neq 0
\end{cases}$

Двойственная задача:

Максимизировать $-\mu^Tb$ при условиях $\begin{cases}
c^T - \lambda^T + \mu^TA = 0\\
\lambda \ge 0
\end{cases}$

In [1]:
import numpy as np
import scipy as sc
import cvxpy
import scipy.linalg
import matplotlib.pyplot as plt
from sympy import *

In [2]:
def minimize(A, b, c):
    mu = cvxpy.Variable(A.shape[0])
    lambda_ = cvxpy.Variable(c.shape[0])
    cvxpy.Problem(cvxpy.Maximize(-mu.T @ b), [c.T - lambda_.T + mu.T @ A == 0, lambda_ >= 0]).solve()
    return mu.value, lambda_.value

mu, lambda_ = minimize(np.array([[1, 2],
                                 [3, 6]]),
                       np.array([5, 15]),
                       np.array([2, 1]))
print('Mu:    ', mu)
print('Lambda:', lambda_)

Mu:     [-0.21588253 -0.09470582]
Lambda: [ 1.50000000e+00 -1.51466043e-09]


### b)

Для всех $\lambda_i \neq 0$ должно выполняться $x_i = 0$, а для остальных можно решить оставшуюся систему $Ax = b$.

In [3]:
def solve_Ax_b(A, b):
    x = symbols(['x' * (i + 1) for i in range(A.shape[1])])
    system = [Eq(sum([A[i, j] * x[j] for j in range(A.shape[1])]), b[i]) for i in range(A.shape[0])]
    soln = solve(system, x)
    return list(soln.values())

def solve_(A, b, c, mu, lambda_):
    lambda_zeros = np.isclose(lambda_, 0)
    A_ = A[:, lambda_zeros]
    res = np.zeros(c.shape)
    res[lambda_zeros] = solve_Ax_b(A_, b)
    return res

solve_(np.array([[1, 2],
                 [3, 6]]),
       np.array([5, 15]),
       np.array([2, 1]), mu, lambda_)

array([0. , 2.5])

# Задание 2

In [4]:
def find_min_radius(x):
    # Я не смогла в какой-нибудь np.repeat, так что будем минимизировать по переменной 'c' размера nXm, где у 'c' все строки равные.
    c = cvxpy.Variable(x.shape)
    problem = cvxpy.Problem(cvxpy.Minimize(
        cvxpy.max(cvxpy.sum((x - c) ** 2, axis=1))
    ), [c[i] == c[i + 1] for i in range(x.shape[0] - 1)])
    problem.solve()
    print('Center:', c.value[0])
    print('Dist:  ', np.sqrt(problem.value))

find_min_radius(np.array([[0, 5],
                          [3, 4],
                          [-4, 3],
                          [-5, 0],
                          [5, 0],
                          [-4, -3]]))

find_min_radius(np.array([[0, 5, 1, 4, 5, 2],
                          [0, 3, 1, 4, 5, 2]]))

Center: [-9.14144754e-09  2.85431219e-08]
Dist:   4.99999999316947
Center: [0. 4. 1. 4. 5. 2.]
Dist:   0.9999999996147417


# Задание 3

Максимизируем $\min\limits_{y_i = 0} a^T x_i - \max\limits_{y_i = 1} a^T x_i$

In [5]:
def find_solution(x, y):
    a = cvxpy.Variable((1, x.shape[1]))
    min_ = cvxpy.Variable(1)
    max_ = cvxpy.Variable(1)
    problem = cvxpy.Problem(cvxpy.Maximize(min_ - max_),
                            [cvxpy.sum_squares(a) <= 1] + 
                            [a @ x0 >= min_ for x0 in x[y == 0]] + 
                            [a @ x1.T <= max_ for x1 in x[y == 1]])
    problem.solve()
    print('a:    ', a.value)
    print('value:', problem.value)

find_solution(np.array([[1, 2], [2, 1], [1, 1]]), np.array([0, 0, 1]))

a:     [[0.70710678 0.70710678]]
value: 0.7071067811800713
